In [24]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [25]:
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)
col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [26]:
for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576
2,8,4,20,15,445,0.99,0.99,398.1,7962,398
3,9,1,5,20,355,1.03,0.8,396.75,7935,472
4,8,5,10,15,350,0.9,0.91,390.25,7805,468
5,8,1,10,16,345,0.9,0.91,387.9,7758,465
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,272
68,17,1,23,22,565,1.03,0.8,289.55,5791,334
69,19,5,30,10,475,0.95,0.92,289.1,5782,311
70,6,5,15,5,250,1,0.92,288.9,5778,308


In [55]:
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    #Setting Parameter

    c1 = 2
    c2 = 2
    r1 = (0, 1)
    r2 = (0, 1)
    iteration_max = 10 #40
    particles = 10 #70
    fitness_value = 200
    productivity = 20
    inertia_max = 0.9
    inertia_min = 0.4
    inertia = 0
    temp = 0
    ae = 0
    particles_position = [0]

    #perhitungan UCP
    def ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf):
        ucSimple = xSimple * simpleUC
        ucAverage = xAverage * averageUC
        ucComplex = xComplex * complexUC
    
        uucw = ucSimple + ucAverage + ucComplex
        uucp = uaw + uucw
        return uucp * tcf * ecf

    #Generate random Simple Use Case Complexity Weight Parameter
    def randomSimpleUCW(lower_bound=5, upper_bound=7.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Average Use Case Complexity Weight Parameter 
    def randomAverageUCW(lower_bound=7.5, upper_bound=12.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Complex Use Case Complexity Weight Parameter
    def randomComplexUCW(lower_bound=12.5, upper_bound=15, n_particles=10):
        #inisialisasi posisi awal
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]
    
    swarm_size = 5 #70 
    simpleUC = col1.loc[index,"S"]
    averageUC = col1.loc[index,"A"]
    complexUC = col1.loc[index,"C"]
    estimated_effort = 0
    uaw = col1.loc[index,"UAW"]
    uucw = col1.loc[index,"UUCW"]
    tcf = col1.loc[index,"TCF"]
    ecf = col1.loc[index,"ECF"]
    real_effort = col1.loc[index,"Real_Effort"]
    array_aeParticle = []
    ae_minimum = 5000
    
    for i in range(iteration_max):
        inertia = inertia_max - ((inertia_max-inertia_min)*i/iteration_max)
        print('Perulangan ',i)
        print('inertia :', inertia)
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        print('r1 :', r1)
        print('r2 :', r2)
        print()
        
        for j in range(swarm_size):    
                
            xSimple = randomSimpleUCW()
            xAverage = randomAverageUCW()
            xComplex = randomComplexUCW()
            case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
            estimated_effort = case * 20
        
            #Selisih estimasi effort
            ae = abs(estimated_effort - real_effort)
            
            if ae_minimum > ae:
                ae_minimum = ae
                letak_i = i
                letak_j = j
                xSimpleMin = xSimple
                xAverageMin = xAverage
                xComplexMin = xComplex
            
            #Memasukkan selisih
            #array_aeParticle.append(ae)
            #aeParticle_min = np.min(array_aeParticle)
            #index_of_minimum = np.where(array_aeParticle == aeParticle_min)
        
            #menjumlah 10 angka selisih
            temp = np.array(temp+ae)
         
            #print('Partikel ke', j,':',estimated_effort)
            #print('ucp   ', case)
            print('ae ', j, ae)
            #print('S     ', xSimple)
            #print('A     ', xAverage)
            #print('C     ', xComplex)
            #print()
            
            #print('estimated effort', estimated_effort)
            
            if j == 0:
                pbest_simple = xSimple
                pbest_average = xAverage
                pbest_complex = xComplex
                
                vSimple = random.uniform(0, 1)
                vAverage = random.uniform(0, 1)
                vComplex = random.uniform(0, 1)
                
                xSimple = pbest_simple
                xAveragepbest_average = pbest_average
                xComplex = pbest_complex
        
        #print('ae terkecil : ',np.min(aeParticle_min))
        #print('letak ae : ',index_of_minimum)
        #print('pada perulangan ke :', aeParticle_min.index(min(aeParticle_min)))
        #mae = temp/swarm_size
        #print('MAE :', mae)
        print()
        array_aePerulangan.append(aeParticle_min)
    print('ae_minimum', ae_minimum)
    print('i', letak_i)
    print('j', letak_j)
    print('S Min', xSimpleMin)
    print('A Min', xAverageMin)
    print('C Min', xComplexMin)
    #print('ae terkecil keseluruhan : ',np.min(array_aePerulangan))
    #print('letak ae : ',index_of_minimum)


____ DATA  1 ____
Perulangan  0
inertia : 0.9
r1 : 0.9811176837533575
r2 : 0.7626146672099673

ae  0 3597.0625813817715
ae  1 3347.331376791245
ae  2 3299.058576873518
ae  3 3099.9411439072755
ae  4 3575.5909770123753

Perulangan  1
inertia : 0.85
r1 : 0.2236683741860791
r2 : 0.8610240161288771

ae  0 2790.771985255984
ae  1 3029.6048503310103
ae  2 3105.9575118782222
ae  3 3215.769478787882
ae  4 3423.892039866216

Perulangan  2
inertia : 0.8
r1 : 0.27153799795215294
r2 : 0.7591094906852447

ae  0 3554.21652185121
ae  1 3167.525887083518
ae  2 3600.7782128630342
ae  3 3085.808941558079
ae  4 2945.89424635673

Perulangan  3
inertia : 0.75
r1 : 0.4255752768704032
r2 : 0.807304776328091

ae  0 2984.6825041103502
ae  1 3309.671794939157
ae  2 3610.4646614498324
ae  3 2995.685923309047
ae  4 3292.2157718928356

Perulangan  4
inertia : 0.7
r1 : 0.513775229475934
r2 : 0.8372071578022631

ae  0 3232.1836154146113
ae  1 3257.665333378626
ae  2 3222.63585918816
ae  3 2915.7998303698296
ae  4 3

____ DATA  7 ____
Perulangan  0
inertia : 0.9
r1 : 0.6069352224291361
r2 : 0.11931089346009349

ae  0 102.80539124199731
ae  1 134.81540879048407
ae  2 469.5028203256461
ae  3 1544.9774411197777
ae  4 681.7248167506314

Perulangan  1
inertia : 0.85
r1 : 0.6960097598368997
r2 : 0.8494671804824926

ae  0 695.0552481106424
ae  1 70.14738461470552
ae  2 684.6916824267246
ae  3 52.04998721162883
ae  4 542.472480641024

Perulangan  2
inertia : 0.8
r1 : 0.5598329157310785
r2 : 0.7478550808093486

ae  0 91.3295614501767
ae  1 493.9250017498971
ae  2 623.5873733454755
ae  3 222.74393642055384
ae  4 229.2856543572616

Perulangan  3
inertia : 0.75
r1 : 0.3660449778948517
r2 : 0.5128349795825426

ae  0 843.8615568406422
ae  1 34.13234791571267
ae  2 92.20345516994712
ae  3 806.2458246896531
ae  4 114.83707843419052

Perulangan  4
inertia : 0.7
r1 : 0.23032837143037688
r2 : 0.8740760771443078

ae  0 986.9297023293311
ae  1 1189.4926090770605
ae  2 541.2017920455073
ae  3 1220.3712652922677
ae  4 67

ae  1 3045.1731200512395
ae  2 2921.2216925272623
ae  3 2866.8571610068184
ae  4 2533.014072624589

Perulangan  2
inertia : 0.8
r1 : 0.9360591488098101
r2 : 0.06345100505939716

ae  0 2426.085361264035
ae  1 2863.7705521621956
ae  2 2504.979989341269
ae  3 2462.4542316333727
ae  4 2554.3756499194533

Perulangan  3
inertia : 0.75
r1 : 0.45216169280964225
r2 : 0.8222199927779524

ae  0 2810.3166351948303
ae  1 2854.019929779359
ae  2 2799.029971056397
ae  3 2537.2265744392607
ae  4 2172.409492649539

Perulangan  4
inertia : 0.7
r1 : 0.831809554732728
r2 : 0.5331481925869771

ae  0 2983.6200334780624
ae  1 2194.8927042417918
ae  2 3091.863256145444
ae  3 2323.0528236069695
ae  4 2094.8686197152238

Perulangan  5
inertia : 0.65
r1 : 0.4817817682627339
r2 : 0.7410501367072506

ae  0 2807.979881262032
ae  1 2496.4653972448314
ae  2 2842.970864505767
ae  3 2547.175295007324
ae  4 2518.2734020297657

Perulangan  6
inertia : 0.6000000000000001
r1 : 0.9814709918043922
r2 : 0.17704713791396187

a

ae  1 1729.0857730075313
ae  2 1136.5550916789498
ae  3 1500.1178726284052
ae  4 1342.514289543863

Perulangan  1
inertia : 0.85
r1 : 0.9758120802382299
r2 : 0.7638976394993352

ae  0 1799.3761308846279
ae  1 1584.1089234617803
ae  2 1156.1282576837539
ae  3 1881.1593393878939
ae  4 2026.9192586519312

Perulangan  2
inertia : 0.8
r1 : 0.244663241466071
r2 : 0.020542002525556713

ae  0 2027.9053933885334
ae  1 1097.948138716607
ae  2 1537.9556550571015
ae  3 1371.7887419507624
ae  4 1864.4151508983068

Perulangan  3
inertia : 0.75
r1 : 0.24678975533592784
r2 : 0.013808596565668019

ae  0 2062.868453248544
ae  1 2073.325573124177
ae  2 898.2526754776427
ae  3 1096.272472323095
ae  4 1656.7416331284812

Perulangan  4
inertia : 0.7
r1 : 0.30851188707535404
r2 : 0.5237353499609767

ae  0 1425.9575277035392
ae  1 1715.2630007766356
ae  2 2239.4702842553806
ae  3 1356.327068525321
ae  4 1144.3858405608898

Perulangan  5
inertia : 0.65
r1 : 0.961768728786666
r2 : 0.23132475650988737

ae  0 210

ae  4 2419.187672414916

Perulangan  4
inertia : 0.7
r1 : 0.7525891613909209
r2 : 0.5135037218000433

ae  0 2455.858976640875
ae  1 2267.592177259964
ae  2 2643.580044544915
ae  3 1937.2247943239136
ae  4 2450.930800053562

Perulangan  5
inertia : 0.65
r1 : 0.062239907799554905
r2 : 0.39286428921578254

ae  0 2136.0225411089004
ae  1 2526.820390527527
ae  2 2352.6084333175822
ae  3 2502.3137626881216
ae  4 2768.9537454267247

Perulangan  6
inertia : 0.6000000000000001
r1 : 0.8604841455139111
r2 : 0.6437866303817044

ae  0 2239.4187323077113
ae  1 2549.1031062923303
ae  2 1963.1692479754338
ae  3 2919.6619349785688
ae  4 1981.609959414488

Perulangan  7
inertia : 0.55
r1 : 0.2746573402950627
r2 : 0.43823887796853533

ae  0 2267.6693732607437
ae  1 2185.8307677038074
ae  2 2601.6203829107726
ae  3 2278.361522784472
ae  4 2419.1913966669536

Perulangan  8
inertia : 0.5
r1 : 0.6115750765738835
r2 : 0.9078829112819763

ae  0 2031.2698925798904
ae  1 2564.4317487371954
ae  2 2262.96018729632

...

ae  4 1130.8571821120095

Perulangan  1
inertia : 0.85
r1 : 0.5745174435482542
r2 : 0.8853460282255818

ae  0 2188.73330769914
ae  1 1415.2380665079017
ae  2 1296.8362678180247
ae  3 1413.5557815484162
ae  4 1925.3834948855456

Perulangan  2
inertia : 0.8
r1 : 0.17702988190483837
r2 : 0.9227155386469155

ae  0 1353.4729567095183
ae  1 713.3263652542655
ae  2 977.0596552769239
ae  3 1650.4563456085943
ae  4 1549.3139538063715

Perulangan  3
inertia : 0.75
r1 : 0.7989312729065803
r2 : 0.8897797370003034

ae  0 1421.4783038211817
ae  1 1737.8423170480764
ae  2 771.1258801310023
ae  3 976.5412817931574
ae  4 1211.7178007083394

Perulangan  4
inertia : 0.7
r1 : 0.19951320511634973
r2 : 0.7297768771329456

ae  0 1301.6922036346614
ae  1 1061.1121359621711
ae  2 1107.406718614201
ae  3 2290.871163619344
ae  4 1246.242645996539

Perulangan  5
inertia : 0.65
r1 : 0.9582429681412574
r2 : 0.965736055685241

ae  0 1102.907851284721
ae  1 1374.1740212652767
ae  2 1234.0361842282655
ae  3 1956.71389

r2 : 0.05965620214491163

ae  0 846.4893853335743
ae  1 196.3072613341801
ae  2 759.395412877464
ae  3 202.86565509635238
ae  4 58.75039165823728

Perulangan  9
inertia : 0.45
r1 : 0.8932540650134957
r2 : 0.2923470089845962

ae  0 848.3980752122625
ae  1 118.24786117215899
ae  2 517.1727378740725
ae  3 304.7438255495781
ae  4 35.57743074048267

ae_minimum 15.067180791522333
i 6
j 0
S Min 6.467360283191159
A Min 9.446899224526438
C Min 14.35051419244608

____ DATA  54 ____
Perulangan  0
inertia : 0.9
r1 : 0.9154804750218888
r2 : 0.11996799969695904

ae  0 346.51778180758174
ae  1 114.01889527791081
ae  2 464.21729251409306
ae  3 732.835187525503
ae  4 1122.4165606534043

Perulangan  1
inertia : 0.85
r1 : 0.07484318235874587
r2 : 0.34658313417497877

ae  0 248.42443921068207
ae  1 1326.7475559317008
ae  2 164.88383676728517
ae  3 183.84983486245437
ae  4 1085.007515626282

Perulangan  2
inertia : 0.8
r1 : 0.6623719741427707
r2 : 0.9202324744729533

ae  0 1286.4867388220937
ae  1 880.0195

r1 : 0.3711171975795605
r2 : 0.036228323243779115

ae  0 593.1715446729813
ae  1 681.1511316456863
ae  2 437.74460953638754
ae  3 1699.743212525419
ae  4 1660.931355628234

ae_minimum 365.6368178007033
i 3
j 3
S Min 5.24080288754118
A Min 12.217528208124037
C Min 14.43414773629377

____ DATA  58 ____
Perulangan  0
inertia : 0.9
r1 : 0.5419802643163926
r2 : 0.2327790907257773

ae  0 1653.945739691244
ae  1 1567.7763986057098
ae  2 1132.8843501901265
ae  3 995.7991286360466
ae  4 843.7294999902942

Perulangan  1
inertia : 0.85
r1 : 0.5930446449470511
r2 : 0.29069068977194734

ae  0 830.0067619582132
ae  1 1696.1803782787902
ae  2 1681.9508363527402
ae  3 1919.5915956967383
ae  4 1856.3932447076077

Perulangan  2
inertia : 0.8
r1 : 0.7115069674277505
r2 : 0.28832368601658465

ae  0 2094.6113755994375
ae  1 1087.3546279348975
ae  2 1119.940181223209
ae  3 1370.7441223081687
ae  4 1058.264814182021

Perulangan  3
inertia : 0.75
r1 : 0.5657951883788023
r2 : 0.5142574864127653

ae  0 1227.455

ae_minimum 9.559207728976617
i 3
j 4
S Min 5.264315577695353
A Min 9.574270944973193
C Min 14.153666530756068

____ DATA  63 ____
Perulangan  0
inertia : 0.9
r1 : 0.5258832266245125
r2 : 0.013773250079261112

ae  0 1395.027133202817
ae  1 1528.1819792223941
ae  2 1320.5227421785248
ae  3 1390.126648276333
ae  4 1623.346635791041

Perulangan  1
inertia : 0.85
r1 : 0.7020414054956258
r2 : 0.026885420873033694

ae  0 846.8393234281557
ae  1 913.7296750572532
ae  2 747.1235027799685
ae  3 1053.3910616338799
ae  4 1415.7312392599233

Perulangan  2
inertia : 0.8
r1 : 0.38590814899045545
r2 : 0.8739611691656622

ae  0 1531.9276166693853
ae  1 905.1249798508898
ae  2 1125.9472128143516
ae  3 1329.1966627002657
ae  4 909.0064777512653

Perulangan  3
inertia : 0.75
r1 : 0.016427241334678877
r2 : 0.4803951409395101

ae  0 1166.3221788716
ae  1 948.892850642269
ae  2 1038.4795317307717
ae  3 881.7251782678995
ae  4 1112.396477865319

Perulangan  4
inertia : 0.7
r1 : 0.6597735411768096
r2 : 0.15326

inertia : 0.9
r1 : 0.7510983797562559
r2 : 0.3812446815668947

ae  0 2602.3234580464414
ae  1 3970.5378094434454
ae  2 2408.9266306298514
ae  3 3482.6059534498
ae  4 2476.340446428225

Perulangan  1
inertia : 0.85
r1 : 0.810505791849154
r2 : 0.6052447627407226

ae  0 2375.102995919404
ae  1 2276.5600909519453
ae  2 1776.4618699556813
ae  3 3485.8260011062557
ae  4 2820.575241480936

Perulangan  2
inertia : 0.8
r1 : 0.9101203940688687
r2 : 0.027317683675856297

ae  0 1493.2943205854672
ae  1 3289.121764743446
ae  2 1910.3968912757828
ae  3 2978.5048231729616
ae  4 1598.1973758451732

Perulangan  3
inertia : 0.75
r1 : 0.13036739363779093
r2 : 0.8929663975145954

ae  0 2939.37947763834
ae  1 2477.968494638637
ae  2 2107.259070715938
ae  3 2811.9569856567996
ae  4 3380.2051980774704

Perulangan  4
inertia : 0.7
r1 : 0.19142054072764336
r2 : 0.4091495319087428

ae  0 2882.0557426969663
ae  1 1428.5992456902968
ae  2 3110.294085903737
ae  3 2468.6881607391697
ae  4 1637.0801241482632

Perula